In [ ]:
import h5py
import numpy as np

Gives the bbox and label for n-th box

In [ ]:
f = h5py.File("data/train_digitStruct.mat", "r")
bbox_refs = f["digitStruct"]["bbox"]
names = f["digitStruct"]["name"]

def read_attr(attr):
    """Read a bbox attribute (left/top/width/height/label)."""
    # If it's a single reference
    if len(attr) == 1:
        return attr[0][0]
    # If multiple references
    return np.array([a[0] for a in attr]).astype(float)

def read_string(ref):
    return "".join([chr(c[0]) for c in f[ref]])

def get_name(i):
    name_ref = names[i-1][0]
    return read_string(name_ref)

def get_boxes(img_num):
    master_ref = bbox_refs[img_num-1][0]
    bbox_data_struct = f[master_ref]
    label_refs = bbox_data_struct["label"]
    num_of_digits = label_refs.shape[0]

    def get_field(field, digit_num):
        field_refs = bbox_data_struct[field]

        # if only 1 digit in img then refrence is to float
        if (field_refs.dtype == np.dtype("float64")):
            field_value = field_refs[0][0]
        else: # otherwise have to get the that the specific digit_num
            field_value = f[field_refs[digit_num][0]][0][0]

        return field_value

    img_data = []
    for i in range(num_of_digits):
        digit_dict = {
            "left": get_field("left", i),
            "top": get_field("top", i),
            "height": get_field("height", i),
            "width": get_field("width", i),
            "label": get_field("label", i),
        }
        img_data.append(digit_dict)

    return img_data


bbox_num = 1
filename = get_name(bbox_num)
box_info = get_boxes(bbox_num)
print(filename, box_info)


In [ ]:
from PIL import Image
import numpy as np
image = Image.open('data/train/1.png')
image_array = np.array(image)

list = []
names = []

for i in range(1,100):
    name = str(i) + ".png"
    path = 'data/train/' + name
    image = Image.open(path).convert('L')
    image_array = np.array(image) / 255 #shape(height,width,3) - 3 is RGB.
    list.append(image_array)
    names.append(name)
npArray = np.array(list, dtype='object')
# print(npArray.shape)
np.save("data/train_image_data.npy", npArray)

npArray[0][0]
names



In [ ]:

#Array of bounding boxes for each image
#TODO maybe each array element should have a value for image name.
bbox_array = []
for i in range(1,len(names)+1):
    filename = get_name(i)
    box_info = get_boxes(i)
    info_array = []
    for info in box_info:
        arr = [info["left"], info["top"], info["height"], info["width"]]
        info_array.append(arr)
    bbox_array.append(info_array)
nArr = np.array(bbox_array, dtype="object")
np.save("data/train_bbox.npy", nArr)





